In [1]:
import tensorflow as tf
import keras
from keras import layers
import os
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [9]:
# this sampling layer is the bottleneck layer of variational autoencoder,
# it uses the output from two dense layers z_mean and z_log_var as input, 
# convert them into normal distribution and pass them to the decoder layer
 
class Sampling(layers.Layer):
    """Uses (mean, log_var) to sample z, the vector encoding a digit."""
 
    def call(self, inputs):
        mean, log_var = inputs
        batch = tf.shape(mean)[0]
        dim = tf.shape(mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        z = mean + tf.exp(0.5 * log_var) * epsilon
        print("Z: ", tf.reduce_min(z), "|", tf.reduce_max(z), 
              "__", "mean: ", tf.reduce_min(mean), "|", tf.reduce_max(mean), 
              "__", "log_var: ", tf.reduce_min(log_var), "|", tf.reduce_max(log_var),  
              "__", "epsilon: ", tf.reduce_min(epsilon), "|", tf.reduce_max(epsilon))
        return z
    


class VAE(keras.Model):
    def __init__(self, input_shape, conv_filters, conv_kernels, conv_strides,
                 latent_space_dim, **kwargs):
        super().__init__(**kwargs)
        self.encoder_input_shape = input_shape # [28, 28, 1]
        self.conv_filters = conv_filters # [2, 4, 8]
        self.conv_kernels = conv_kernels # [3, 5, 3]
        self.conv_strides = conv_strides # [1, 2, 2]
        self.latent_space_dim = latent_space_dim # 2
        self.reconstruction_loss_weight = 1000000

        self.encoder = None
        self.decoder = None

        self._num_conv_layers = len(conv_filters)
        self._shape_before_bottleneck = None
        self._model_input = None

        self._build_model()

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
 
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
 
    def train_step(self, data):
        with tf.GradientTape() as tape:
            mean,log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + log_var - tf.square(mean) - tf.exp(log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def reconstruct(self, images):
        latent_representations = self.encoder.predict(images)
        reconstructed_images = self.decoder.predict(latent_representations)
        return reconstructed_images, latent_representations
    
    def _build_model(self):
        ## Build encoder
        self._model_input = keras.Input(shape=self.encoder_input_shape) 
        # add conv layers 
        x = self._model_input
        for layer_index in range(self._num_conv_layers):
            x = layers.Conv2D(filters=self.conv_filters[layer_index],
                                kernel_size=self.conv_kernels[layer_index],
                                activation="relu",
                                strides=self.conv_strides[layer_index],
                                padding="same")(x)
            x = layers.BatchNormalization()(x)
        # create bottleneck
        self._shape_before_bottleneck = K.int_shape(x)[1:]
        x = layers.Flatten()(x)
        x = layers.Dense(16, activation="relu")(x)
        mean = layers.Dense(self.latent_space_dim, name="mean")(x)
        log_var = layers.Dense(self.latent_space_dim, name="log_var")(x)
        z = Sampling()([mean, log_var])
        
        self.encoder = keras.Model(self._model_input, [mean, log_var, z], name="encoder")
        self.encoder.summary()

        ## Build decoder
        latent_inputs = keras.Input(shape=(self.latent_space_dim,))
        num_neurons = np.prod(self._shape_before_bottleneck)
        x = layers.Dense(num_neurons, activation="relu")(latent_inputs) #64 * 16 * 64
        x = layers.Reshape(self._shape_before_bottleneck)(x) # use shape before bottleneck
        # add conv transpose layers
        for layer_index in reversed(range(1, self._num_conv_layers)):
            x = layers.Conv2DTranspose(filters=self.conv_filters[layer_index],
                                        kernel_size=self.conv_kernels[layer_index],
                                        activation="relu",
                                        strides=self.conv_strides[layer_index],
                                        padding="same")(x)
            x = layers.BatchNormalization()(x)
        decoder_outputs = layers.Conv2DTranspose(1, 
                                                kernel_size=self.conv_kernels[0],
                                                activation="sigmoid", 
                                                strides=self.conv_strides[0], 
                                                padding="same")(x)
        self.decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        self.decoder.summary()

    @classmethod
    def load(cls, save_folder="model_vae"):
        parameters_path = os.path.join(save_folder, "parameters.pkl")
        with open(parameters_path, "rb") as f:
            parameters = pickle.load(f)
        autoencoder = VAE(*parameters)
        weights_path = os.path.join(save_folder, "weights.h5")
        autoencoder.load_weights(weights_path)
        return autoencoder



In [3]:
def load_fsdd(spectograms_path):
    x_train = []
    for root, _, file_names in os.walk(spectograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectogram = np.load(file_path) # ex. (n_bins, n_frames) --> abbiamo bisogno di 3 dimensioni (3 channels)-> (n_bins, n_frames, 1)
            x_train.append(spectogram)

    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    #x_train = tf.data.Dataset.from_tensor_slices(x_train)
    return x_train


def plot_latent_space(vae, n=10, figsize=5):
    # display a n*n 2D manifold of images
    img_size = 28
    scale = 0.5
    figure = np.zeros((img_size * n, img_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of images classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]
 
    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(sample, verbose=0)
            images = x_decoded[0].reshape(img_size, img_size)
            figure[
                i * img_size : (i + 1) * img_size,
                j * img_size : (j + 1) * img_size,
            ] = images
 
    plt.figure(figsize=(figsize, figsize))
    start_range = img_size // 2
    end_range = n * img_size + start_range
    pixel_range = np.arange(start_range, end_range, img_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()



In [4]:
# spectrogram input
SPECTOGRAMS_PATH = os.path.join("dataset", "fsdd", "spectrograms")
x_train = load_fsdd(SPECTOGRAMS_PATH)
with open(os.path.join('dataset', 'fsdd', 'pickled_spectrograms.npy'), 'wb') as f:
    np.save(f, x_train)

In [5]:
with open(os.path.join('dataset', 'fsdd', 'pickled_spectrograms.npy'), 'rb') as f:
    x_train = np.load(f)

In [6]:

nan_count = np.count_nonzero(np.isnan(x_train))

In [7]:
print(nan_count)

0


In [10]:
tf.config.run_functions_eagerly(True)

vae = VAE(
        input_shape=(256, 64, 1),
        conv_filters=(512, 256, 128, 64, 32),
        conv_kernels=(3, 3, 3, 3, 3),
        conv_strides=(2, 2, 2, 2, (2, 1)),
        latent_space_dim=128
    )
vae.compile(optimizer=keras.optimizers.Adam(clipnorm=1.0))
#vae.fit(fashion_mnist, epochs=10, batch_size=128)
vae.fit(x_train, epochs=1, batch_size=32)

WEIGHTS_Path = os.path.join("model", "trial.weights.h5")
vae.save_weights(WEIGHTS_Path)


Z:  Tensor("Min:0", shape=(), dtype=float32) | Tensor("Max:0", shape=(), dtype=float32) __ mean:  Tensor("Min_1:0", shape=(), dtype=float32) | Tensor("Max_1:0", shape=(), dtype=float32) __ log_var:  Tensor("Min_2:0", shape=(), dtype=float32) | Tensor("Max_2:0", shape=(), dtype=float32) __ epsilon:  Tensor("Min_3:0", shape=(), dtype=float32) | Tensor("Max_3:0", shape=(), dtype=float32)


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 64,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 32,   │      5,120 │ input_layer[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 32,   │      2,048 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 16,    │  1,179,904 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 16,    │      1,024 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 8,     │    295,040 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 8,     │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 16, 4, 64) │     73,792 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 4, 64) │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 8, 4, 32)  │     18,464 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 4, 32)  │        128 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1024)      │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │     16,400 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Dense)        │ (None, 128)       │      2,176 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ log_var (Dense)     │ (None, 128)       │      2,176 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling (Sampling) │ (None, 128)       │          0 │ mean[0][0],       │
│                     │                   │            │ log_var[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,597,040 (6.09 MB)

 Trainable params: 1,595,056 (6.08 MB)

...

 Total params: 533,089 (2.03 MB)

 Trainable params: 532,129 (2.03 MB)

 Non-trainable params: 960 (3.75 KB)

c:\Users\janas\anaconda3\envs\musicmodel\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import librosa

from preprocess import MinMaxNormaliser

class SoundGenerator:
    """SoundGenerator is responsible for generating audio spectograms.
    """

    def __init__(self, vae, hop_length):
        self.vae = vae
        self.hop_length = hop_length
        self._min_max_normalizer = MinMaxNormaliser(0, 1)

    def generate(self, spectograms, min_max_values):
        generated_spectograms, latent_representations = \
            self.vae.reconstruct(spectograms)
        signals = self.convert_spectograms_to_audio(generated_spectograms, min_max_values)
        return signals, latent_representations

    def convert_spectograms_to_audio(self, spectograms, min_max_values):
        signals = []
        for spectogram, min_max_value in zip(spectograms, min_max_values):
            # reshape the log spectogram
            log_spectogram = spectogram[:, :, 0] # elimina la dimensione 3

            # apply denormalization
            denorm_log_spec = self._min_max_normalizer.denormalise(log_spectogram,
                                                                   min_max_value["min"],
                                                                   min_max_value["max"])

            # log spectogram -> spectogram
            spec = librosa.db_to_amplitude(denorm_log_spec)

            # apply Griffin-Lim
            signal = librosa.istft(spec, hop_length=self.hop_length)

            # append signal to "signals"
            signals.append(signal)

        return signals

In [ ]:
def load_fsdd(spectrograms_path):
    x_train = []
    file_paths = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram)
            file_paths.append(file_path)
    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    return x_train, file_paths


def select_spectrograms(spectrograms,
                        file_paths,
                        min_max_values,
                        num_spectrograms=2):
    sampled_indexes = np.random.choice(range(len(spectrograms)), num_spectrograms)
    sampled_spectrogrmas = spectrograms[sampled_indexes]
    file_paths = [file_paths[index] for index in sampled_indexes]
    sampled_min_max_values = [min_max_values[file_path] for file_path in
                           file_paths]
    print(file_paths)
    print(sampled_min_max_values)
    return sampled_spectrogrmas, sampled_min_max_values


def save_signals(signals, save_dir, sample_rate=22050):
    for i, signal in enumerate(signals):
        save_path = os.path.join(save_dir, str(i) + ".wav")
        sf.write(save_path, signal, sample_rate)

In [ ]:
HOP_LENGTH = 256
SAVE_DIR_ORIGINAL = os.path.join('dataset', 'generation', 'original')
SAVE_DIR_GENERATED = os.path.join('dataset', 'generation', 'original')
MIN_MAX_VALUES_PATH = os.path.join('dataset', 'fsdd', 'min_max_values.pkl')

vae = VAE.load("model_vae")
sound_generator = SoundGenerator(vae, HOP_LENGTH)

# load spectrograms + min max values
with open(MIN_MAX_VALUES_PATH, "rb") as f:
    min_max_values = pickle.load(f)

specs, file_paths = load_fsdd(SPECTOGRAMS_PATH)

# sample spectrograms + min max values
sampled_specs, sampled_min_max_values = select_spectrograms(specs,
                                                            file_paths,
                                                            min_max_values,
                                                            5)

# generate audio for sampled spectrograms
signals, _ = sound_generator.generate(sampled_specs,
                                        sampled_min_max_values)

# convert spectrogram samples to audio
original_signals = sound_generator.convert_spectograms_to_audio(sampled_specs, sampled_min_max_values)

# save audio signals
save_signals(signals, SAVE_DIR_GENERATED)
save_signals(original_signals, SAVE_DIR_ORIGINAL)

In [ ]:
plot_latent_space(vae)